In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X = np.loadtxt(open("AD_MRI_DATA/2-CN_MCI/2.2-MRI_SurfaceArea.csv","rb"),delimiter=",",skiprows=0)
X = np.array(X, dtype=float)
print("X:" + str(X.shape))

# Load the labels
with open('AD_MRI_DATA/2-CN_MCI/2-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X:(188, 68)
y:(188,)


## 2. Lasso with Classifiers

### 2.1 Lasso

In [3]:
model = linear_model.LassoCV(alphas=[1,0.1,0.01,0.005,0.001,0.0001])
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)
3
selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 1.0
X_selected: (188, 50)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011292196503951857, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.028956739653661856, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005511914588272049, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt

### 2.2 Classifiers

In [4]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")


SVM
[[97  3]
 [88  0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.52      0.97      0.68       100

    accuracy                           0.52       188
   macro avg       0.26      0.48      0.34       188
weighted avg       0.28      0.52      0.36       188

auc: 0.4723202614379086

KNN
[[27 73]
 [25 63]]
              precision    recall  f1-score   support

           0       0.46      0.72      0.56        88
           1       0.52      0.27      0.36       100

    accuracy                           0.48       188
   macro avg       0.49      0.49      0.46       188
weighted avg       0.49      0.48      0.45       188

auc: 0.47947712418300653

Gaussian Naive Bayes
[[48 52]
 [48 40]]
              precision    recall  f1-score   support

           0       0.43      0.45      0.44        88
           1       0.50      0.48      0.49       100

    accuracy                           0.47  

### 2.3 Multi-core SVM

In [5]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.30000000000000004 w2:0.7000000000000001 w3:-1.1102230246251565e-16
score:0.587719298245614


In [6]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[57 43]
 [46 42]]
              precision    recall  f1-score   support

           0       0.49      0.48      0.49        88
           1       0.55      0.57      0.56       100

    accuracy                           0.53       188
   macro avg       0.52      0.52      0.52       188
weighted avg       0.53      0.53      0.53       188

auc: 0.5347385620915033



## 3. ElasticNet with Classifiers

### 3.1 ElasticNet

In [7]:
model = ElasticNetCV(random_state=0,alphas=[1,0.1,0.01,0.005,0.001,0.0001])
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 1.0
X_selected: (188, 22)


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.021240899372806155, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.027536852336787376, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005474374353411804, tolerance: 0.003739333333333334
  model = cd_fast.enet_coordinate_descent_gram(
/Users/zhengwentao/opt

### 3.2 Classifiers

In [8]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
    print("auc: "+str(auc_avg)+"\n")

SVM
[[97  3]
 [87  1]]
              precision    recall  f1-score   support

           0       0.25      0.01      0.02        88
           1       0.53      0.97      0.68       100

    accuracy                           0.52       188
   macro avg       0.39      0.49      0.35       188
weighted avg       0.40      0.52      0.37       188

auc: 0.5283006535947713

KNN
[[26 74]
 [19 69]]
              precision    recall  f1-score   support

           0       0.48      0.78      0.60        88
           1       0.58      0.26      0.36       100

    accuracy                           0.51       188
   macro avg       0.53      0.52      0.48       188
weighted avg       0.53      0.51      0.47       188

auc: 0.5473529411764705

Gaussian Naive Bayes
[[52 48]
 [45 43]]
              precision    recall  f1-score   support

           0       0.47      0.49      0.48        88
           1       0.54      0.52      0.53       100

    accuracy                           0.51   

### 3.3 Multi-core SVM

In [9]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:1.0 w3:0.0
score:0.5263157894736842


In [10]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[100   0]
 [ 88   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.53      1.00      0.69       100

    accuracy                           0.53       188
   macro avg       0.27      0.50      0.35       188
weighted avg       0.28      0.53      0.37       188

auc: 0.5



/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
